In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("Datos_limpios.csv")
df.head(5)

,Unnamed: 0,host_name,host_since,host_response_time,property_type,room_type,bathrooms_text,first_review,last_review,host_response_rate,...,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_since_num,first_review_num,last_review_num
0,0,Rita,26/11/2010,within a few hours,Entire condo,Entire home/apt,1 bath,29/12/2010,30/04/2025,1.0,...,4.78,0,1.0,1.0,0.0,0.0,0.32,417,4827.0,3433.0
1,1,Michelle,19/11/2012,within an hour,Entire condo,Entire home/apt,1 bath,01/02/2013,27/07/2024,1.0,...,4.76,0,1.0,1.0,0.0,0.0,0.46,1141,4827.0,3521.0
2,2,Judy,27/12/2012,within an hour,Private room in home,Private room,1 private bath,29/08/2016,30/09/2023,1.0,...,4.83,0,2.0,0.0,0.0,0.0,0.66,1179,4827.0,3521.0
3,3,Joan,23/04/2013,within an hour,Entire rental unit,Entire home/apt,1 bath,02/06/2013,29/10/2024,1.0,...,4.66,1,9.0,9.0,0.0,0.0,0.65,1296,4827.0,3521.0
4,4,C. F. Sandy,13/06/2013,within a day,Private room in home,Private room,1 shared bath,23/04/2017,09/11/2019,1.0,...,4.53,0,1.0,0.0,0.0,0.0,0.30,1347,4827.0,3521.0


host_response_rate

In [3]:
# --- Encontrar variables más correlacionadas con una variable dependiente ---
# Variable dependiente (puedes cambiarla aquí)
var_dep = "host_response_rate"

# Calculamos todas las correlaciones de esa variable con el resto
correls = df.corr(numeric_only=True)[var_dep].drop(var_dep)   # quitamos la propia variable

# Ordenamos por valor absoluto de mayor a menor
correls_abs = correls.abs().sort_values(ascending=False)

# Mostramos el top 5 (o el número que quieras)
print(f"Variables más correlacionadas con {var_dep}:\n")
print(correls_abs.head(5))

# Si quieres ver también el signo original:
print("\nCon signo original:\n")
print(correls.loc[correls_abs.index[:5]])

Variables más correlacionadas con host_response_rate:

instant_bookable          0.217891
host_since_num            0.143736
number_of_reviews         0.140274
host_is_superhost         0.136555
minimum_maximum_nights    0.135380
Name: host_response_rate, dtype: float64

Con signo original:

instant_bookable         -0.217891
host_since_num           -0.143736
number_of_reviews         0.140274
host_is_superhost         0.136555
minimum_maximum_nights    0.135380
Name: host_response_rate, dtype: float64


In [4]:
#Declaramos las variables dependientes e independientes para la regresión No lineal
Vars_Indep= df[['minimum_maximum_nights']]
Var_Dep= df['host_response_rate']

#Redefinimos las variables 
x= Vars_Indep
y= Var_Dep

Modelo 1 cuadratico

In [5]:
def  func1 (x, a, b, c):
     return a*x**2 + b*x + c

#Ajustamos los parámetros de la función curve_fit
parametros1, covs= curve_fit(func1, df['minimum_maximum_nights'], df['host_response_rate'])

In [6]:
#Obtenemos los coeficientes del modelo de regresión no lineal
parametros1

array([-2.71818975e-08,  3.88534182e-05,  9.88093695e-01])

In [7]:
# Predicciones
y_pred1 = func1(x, *parametros1)

In [8]:
#Calculamos el coeficiente de determinación del modelo
R2_Modelo1 = r2_score(y, y_pred1) 
R2_Modelo1 

0.10636297086776014

In [9]:
#Calculamos el coeficiente de correlación del modelo
R = np.sqrt(R2_Modelo1)
R

0.32613336362255263

Modelo 2 Logaritmico

In [12]:
# Definimos la función logarítmica
def func2(x, a, b):
    return a * np.log(x) + b

In [13]:
# Ajustamos los parámetros de la función curve_fit
# (filtramos x > 0 para evitar errores con log(0))
x = df['minimum_maximum_nights']
x = x[x > 0]
y = df.loc[x.index, 'host_response_rate']
parametros2, covs2 = curve_fit(func2, x, y, maxfev=20000)

In [14]:
# Obtenemos los coeficientes del modelo de regresión no lineal
parametros2

array([0.00272474, 0.98074987])

In [15]:
# Predicciones
y_pred2 = func2(x, *parametros2)

In [16]:
# Calculamos el coeficiente de determinación del modelo
R2_Modelo2 = r2_score(y, y_pred2)
R2_Modelo2

0.18747969008067678

In [17]:
# Calculamos el coeficiente de correlación del modelo
R = np.sqrt(abs(R2_Modelo2))
R

0.4329892493823337

host_acceptance_rate

In [52]:
# --- Encontrar variables más correlacionadas con una variable dependiente ---
# Variable dependiente (puedes cambiarla aquí)
var_dep = "host_acceptance_rate"

# Calculamos todas las correlaciones de esa variable con el resto
correls = df.corr(numeric_only=True)[var_dep].drop(var_dep)   # quitamos la propia variable

# Ordenamos por valor absoluto de mayor a menor
correls_abs = correls.abs().sort_values(ascending=False)

# Mostramos el top 5 (o el número que quieras)
print(f"Variables más correlacionadas con {var_dep}:\n")
print(correls_abs.head(5))

# Si quieres ver también el signo original:
print("\nCon signo original:\n")
print(correls.loc[correls_abs.index[:5]])

Variables más correlacionadas con host_acceptance_rate:

instant_bookable          0.216214
maximum_nights_avg_ntm    0.163894
minimum_maximum_nights    0.163721
maximum_maximum_nights    0.155687
availability_365          0.093106
Name: host_acceptance_rate, dtype: float64

Con signo original:

instant_bookable          0.216214
maximum_nights_avg_ntm    0.163894
minimum_maximum_nights    0.163721
maximum_maximum_nights    0.155687
availability_365          0.093106
Name: host_acceptance_rate, dtype: float64


In [58]:
#Declaramos las variables dependientes e independientes para la regresión No lineal
Vars_Indep= df[['instant_bookable']]
Var_Dep= df['host_acceptance_rate']

#Redefinimos las variables 
x= Vars_Indep
y= Var_Dep

Modelo cuadratico

In [ ]:
def  func1 (x, a, b, c):
     return a*x**2 + b*x + c

#Ajustamos los parámetros de la función curve_fit
parametros1, covs= curve_fit(func1, df['instant_bookable'], df['host_acceptance_rate'])

In [60]:
#Obtenemos los coeficientes del modelo de regresión no lineal
parametros1

array([-148.64514428,  148.67196421,    0.95478261])

In [61]:
# Predicciones
y_pred1 = func1(x, *parametros1)

In [62]:
#Calculamos el coeficiente de determinación del modelo
R2_Modelo1 = r2_score(y, y_pred1) 
R2_Modelo1 

#Calculamos el coeficiente de correlación del modelo
R = np.sqrt(R2_Modelo1)
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo1)
print("R Modelo 1:", R)

R² Modelo 1: 0.04674865599028821
R Modelo 1: 0.21621437507781072


Modelo 2 Logaritmico

In [81]:
# Definimos la función logarítmica
def func2(x, a, b):
    return a * np.log(x) + b

# Ajustamos los parámetros de la función curve_fit
# (filtramos x > 0 para evitar errores con log(0))
x = df['minimum_maximum_nights']
x = x[x > 0]
y = df.loc[x.index, 'host_acceptance_rate']
parametros2, covs2 = curve_fit(func2, x, y, maxfev=20000)

In [82]:
# Obtenemos los coeficientes del modelo de regresión no lineal
parametros2

array([0.00429658, 0.94046204])

In [83]:
# Predicciones
y_pred2 = func2(x, *parametros2)

In [84]:
# Calculamos el coeficiente de determinación del modelo
R2_Modelo2 = r2_score(y, y_pred2)
R2_Modelo2

# Calculamos el coeficiente de correlación del modelo
R = np.sqrt(abs(R2_Modelo2))
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo2)
print("R Modelo 1:", R)

R² Modelo 1: 0.011669275093190268
R Modelo 1: 0.1080244189671496


host_total_listings_count

In [75]:
# --- Encontrar variables más correlacionadas con una variable dependiente ---
# Variable dependiente (puedes cambiarla aquí)
var_dep = "host_total_listings_count"

# Calculamos todas las correlaciones de esa variable con el resto
correls = df.corr(numeric_only=True)[var_dep].drop(var_dep)   # quitamos la propia variable

# Ordenamos por valor absoluto de mayor a menor
correls_abs = correls.abs().sort_values(ascending=False)

# Mostramos el top 5 (o el número que quieras)
print(f"Variables más correlacionadas con {var_dep}:\n")
print(correls_abs.head(5))

# Si quieres ver también el signo original:
print("\nCon signo original:\n")
print(correls.loc[correls_abs.index[:5]])

Variables más correlacionadas con host_total_listings_count:

calculated_host_listings_count                 0.642858
calculated_host_listings_count_entire_homes    0.576308
review_scores_rating                           0.167497
review_scores_value                            0.148337
review_scores_cleanliness                      0.145220
Name: host_total_listings_count, dtype: float64

Con signo original:

calculated_host_listings_count                 0.642858
calculated_host_listings_count_entire_homes    0.576308
review_scores_rating                          -0.167497
review_scores_value                           -0.148337
review_scores_cleanliness                     -0.145220
Name: host_total_listings_count, dtype: float64


In [76]:
#Declaramos las variables dependientes e independientes para la regresión No lineal
Vars_Indep= df[['review_scores_value']]
Var_Dep= df['host_total_listings_count']

#Redefinimos las variables 
x= Vars_Indep
y= Var_Dep

Modelo cuadratico

In [77]:
def  func1 (x, a, b, c):
     return a*x**2 + b*x + c

#Ajustamos los parámetros de la función curve_fit
parametros1, covs= curve_fit(func1, df['review_scores_value'], df['host_total_listings_count'])

#Obtenemos los coeficientes del modelo de regresión no lineal
parametros1

# Predicciones
y_pred1 = func1(x, *parametros1)

In [78]:
#Calculamos el coeficiente de determinación del modelo
R2_Modelo1 = r2_score(y, y_pred1) 
R2_Modelo1 

#Calculamos el coeficiente de correlación del modelo
R = np.sqrt(R2_Modelo1)
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo1)
print("R Modelo 1:", R)

R² Modelo 1: 0.022009194052569403
R Modelo 1: 0.14835495964938078


Logaritmico

In [79]:
# Definimos la función logarítmica
def func2(x, a, b):
    return a * np.log(x) + b

# Ajustamos los parámetros de la función curve_fit
# (filtramos x > 0 para evitar errores con log(0))
x = df['review_scores_value']
x = x[x > 0]
y = df.loc[x.index, 'host_total_listings_count']
parametros2, covs2 = curve_fit(func2, x, y, maxfev=20000)

# Obtenemos los coeficientes del modelo de regresión no lineal
parametros2

# Predicciones
y_pred2 = func2(x, *parametros2)

In [80]:
# Calculamos el coeficiente de determinación del modelo
R2_Modelo2 = r2_score(y, y_pred2)
R2_Modelo2

# Calculamos el coeficiente de correlación del modelo
R = np.sqrt(abs(R2_Modelo2))
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo2)
print("R Modelo 1:", R)

R² Modelo 1: 0.02197798276181595
R Modelo 1: 0.1482497310682753


accommodates

In [85]:
# --- Encontrar variables más correlacionadas con una variable dependiente ---
# Variable dependiente (puedes cambiarla aquí)
var_dep = "accommodates"

# Calculamos todas las correlaciones de esa variable con el resto
correls = df.corr(numeric_only=True)[var_dep].drop(var_dep)   # quitamos la propia variable

# Ordenamos por valor absoluto de mayor a menor
correls_abs = correls.abs().sort_values(ascending=False)

# Mostramos el top 5 (o el número que quieras)
print(f"Variables más correlacionadas con {var_dep}:\n")
print(correls_abs.head(5))

# Si quieres ver también el signo original:
print("\nCon signo original:\n")
print(correls.loc[correls_abs.index[:5]])

Variables más correlacionadas con accommodates:

beds                       0.723959
bedrooms                   0.700020
bathrooms                  0.567192
price                      0.448937
estimated_revenue_l365d    0.193329
Name: accommodates, dtype: float64

Con signo original:

beds                       0.723959
bedrooms                   0.700020
bathrooms                  0.567192
price                      0.448937
estimated_revenue_l365d    0.193329
Name: accommodates, dtype: float64


In [86]:
#Declaramos las variables dependientes e independientes para la regresión No lineal
Vars_Indep= df[['estimated_revenue_l365d']]
Var_Dep= df['accommodates']

#Redefinimos las variables 
x= Vars_Indep
y= Var_Dep

Cuadratico

In [87]:
def  func1 (x, a, b, c):
     return a*x**2 + b*x + c

#Ajustamos los parámetros de la función curve_fit
parametros1, covs= curve_fit(func1, df['estimated_revenue_l365d'], df['accommodates'])

#Obtenemos los coeficientes del modelo de regresión no lineal
parametros1

# Predicciones
y_pred1 = func1(x, *parametros1)

In [88]:
#Calculamos el coeficiente de determinación del modelo
R2_Modelo1 = r2_score(y, y_pred1) 
R2_Modelo1 

#Calculamos el coeficiente de correlación del modelo
R = np.sqrt(R2_Modelo1)
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo1)
print("R Modelo 1:", R)

R² Modelo 1: 0.03738228480764605
R Modelo 1: 0.19334498909370795


Logaritmico

In [ ]:
# Definimos la función logarítmica
def func2(x, a, b):
    return a * np.log(x) + b

# Ajustamos los parámetros de la función curve_fit
# (filtramos x > 0 para evitar errores con log(0))
x = df['estimated_revenue_l365d']
x = x[x > 0]
y = df.loc[x.index, 'accommodates']
parametros2, covs2 = curve_fit(func2, x, y, maxfev=20000)

# Obtenemos los coeficientes del modelo de regresión no lineal
parametros2

# Predicciones
y_pred2 = func2(x, *parametros2)

In [90]:
# Calculamos el coeficiente de determinación del modelo
R2_Modelo2 = r2_score(y, y_pred2)
R2_Modelo2

# Calculamos el coeficiente de correlación del modelo
R = np.sqrt(abs(R2_Modelo2))
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo2)
print("R Modelo 1:", R)

R² Modelo 1: 0.0448343178993138
R Modelo 1: 0.2117411577830673


reviews_per_month

In [91]:
# --- Encontrar variables más correlacionadas con una variable dependiente ---
# Variable dependiente (puedes cambiarla aquí)
var_dep = "reviews_per_month"

# Calculamos todas las correlaciones de esa variable con el resto
correls = df.corr(numeric_only=True)[var_dep].drop(var_dep)   # quitamos la propia variable

# Ordenamos por valor absoluto de mayor a menor
correls_abs = correls.abs().sort_values(ascending=False)

# Mostramos el top 5 (o el número que quieras)
print(f"Variables más correlacionadas con {var_dep}:\n")
print(correls_abs.head(5))

# Si quieres ver también el signo original:
print("\nCon signo original:\n")
print(correls.loc[correls_abs.index[:5]])

Variables más correlacionadas con reviews_per_month:

estimated_occupancy_l365d    0.479040
last_review_num              0.349107
estimated_revenue_l365d      0.348447
number_of_reviews            0.226253
Unnamed: 0                   0.225700
Name: reviews_per_month, dtype: float64

Con signo original:

estimated_occupancy_l365d    0.479040
last_review_num              0.349107
estimated_revenue_l365d      0.348447
number_of_reviews            0.226253
Unnamed: 0                   0.225700
Name: reviews_per_month, dtype: float64


In [93]:
#Declaramos las variables dependientes e independientes para la regresión No lineal
Vars_Indep= df[['estimated_occupancy_l365d']]
Var_Dep= df['reviews_per_month']

#Redefinimos las variables 
x= Vars_Indep
y= Var_Dep

Cuadratico

In [94]:
def  func1 (x, a, b, c):
     return a*x**2 + b*x + c

#Ajustamos los parámetros de la función curve_fit
parametros1, covs= curve_fit(func1, df['estimated_occupancy_l365d'], df['reviews_per_month'])

#Obtenemos los coeficientes del modelo de regresión no lineal
parametros1

# Predicciones
y_pred1 = func1(x, *parametros1)

In [95]:
#Calculamos el coeficiente de determinación del modelo
R2_Modelo1 = r2_score(y, y_pred1) 
R2_Modelo1 

#Calculamos el coeficiente de correlación del modelo
R = np.sqrt(R2_Modelo1)
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo1)
print("R Modelo 1:", R)

R² Modelo 1: 0.2295178470485707
R Modelo 1: 0.47908020941025176


Logaritmico

In [96]:
# Definimos la función logarítmica
def func2(x, a, b):
    return a * np.log(x) + b

# Ajustamos los parámetros de la función curve_fit
# (filtramos x > 0 para evitar errores con log(0))
x = df['estimated_occupancy_l365d']
x = x[x > 0]
y = df.loc[x.index, 'reviews_per_month']
parametros2, covs2 = curve_fit(func2, x, y, maxfev=20000)

# Obtenemos los coeficientes del modelo de regresión no lineal
parametros2

# Predicciones
y_pred2 = func2(x, *parametros2)

In [97]:
# Calculamos el coeficiente de determinación del modelo
R2_Modelo2 = r2_score(y, y_pred2)
R2_Modelo2

# Calculamos el coeficiente de correlación del modelo
R = np.sqrt(abs(R2_Modelo2))
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo2)
print("R Modelo 1:", R)

R² Modelo 1: 0.23300438029549753
R Modelo 1: 0.48270527270322783


price

In [98]:
# --- Encontrar variables más correlacionadas con una variable dependiente ---
# Variable dependiente (puedes cambiarla aquí)
var_dep = "price"

# Calculamos todas las correlaciones de esa variable con el resto
correls = df.corr(numeric_only=True)[var_dep].drop(var_dep)   # quitamos la propia variable

# Ordenamos por valor absoluto de mayor a menor
correls_abs = correls.abs().sort_values(ascending=False)

# Mostramos el top 5 (o el número que quieras)
print(f"Variables más correlacionadas con {var_dep}:\n")
print(correls_abs.head(5))

# Si quieres ver también el signo original:
print("\nCon signo original:\n")
print(correls.loc[correls_abs.index[:5]])

Variables más correlacionadas con price:

bedrooms                   0.518793
bathrooms                  0.502938
accommodates               0.448937
beds                       0.415004
estimated_revenue_l365d    0.162898
Name: price, dtype: float64

Con signo original:

bedrooms                   0.518793
bathrooms                  0.502938
accommodates               0.448937
beds                       0.415004
estimated_revenue_l365d    0.162898
Name: price, dtype: float64


In [99]:
#Declaramos las variables dependientes e independientes para la regresión No lineal
Vars_Indep= df[['estimated_revenue_l365d']]
Var_Dep= df['price']

#Redefinimos las variables 
x= Vars_Indep
y= Var_Dep

Cuadratico

In [100]:
def  func1 (x, a, b, c):
     return a*x**2 + b*x + c

#Ajustamos los parámetros de la función curve_fit
parametros1, covs= curve_fit(func1, df['estimated_revenue_l365d'], df['price'])

#Obtenemos los coeficientes del modelo de regresión no lineal
parametros1

# Predicciones
y_pred1 = func1(x, *parametros1)

In [101]:
#Calculamos el coeficiente de determinación del modelo
R2_Modelo1 = r2_score(y, y_pred1) 
R2_Modelo1 

#Calculamos el coeficiente de correlación del modelo
R = np.sqrt(R2_Modelo1)
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo1)
print("R Modelo 1:", R)

R² Modelo 1: 0.03496821100893199
R Modelo 1: 0.18699789038631423


Logaritmico

In [102]:
# Definimos la función logarítmica
def func2(x, a, b):
    return a * np.log(x) + b

# Ajustamos los parámetros de la función curve_fit
# (filtramos x > 0 para evitar errores con log(0))
x = df['estimated_revenue_l365d']
x = x[x > 0]
y = df.loc[x.index, 'price']
parametros2, covs2 = curve_fit(func2, x, y, maxfev=20000)

# Obtenemos los coeficientes del modelo de regresión no lineal
parametros2

# Predicciones
y_pred2 = func2(x, *parametros2)

In [103]:
# Calculamos el coeficiente de determinación del modelo
R2_Modelo2 = r2_score(y, y_pred2)
R2_Modelo2

# Calculamos el coeficiente de correlación del modelo
R = np.sqrt(abs(R2_Modelo2))
R

# Imprimimos ambos resultados
print("R² Modelo 1:", R2_Modelo2)
print("R Modelo 1:", R)

R² Modelo 1: 0.06421631369286707
R Modelo 1: 0.25340937964658505
